Hidden Markov Model experiment 1

Use the Viterbi Algorithm to estimate the state of mastery given a sequence of content and test interactions

In [3]:
import numpy as np

In [4]:
import pandas as pd

c:\python27\lib\site-packages\pandas\_libs\__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected zd, got zd
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
c:\python27\lib\site-packages\pandas\__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected zd, got zd
  from pandas._libs import (hashtable as _hashtable,
c:\python27\lib\site-packages\pandas\core\dtypes\common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected zd, got zd
  from pandas._libs import algos, lib
c:\python27\lib\site-packages\pandas\core\util\hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected zd, got zd
  from pandas._libs import hashing, tslib
c:\python27\lib\site-packages\pandas\core\indexes\base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected zd, got zd
  f

In [3]:
import networkx as nx

First step:  Create the state space and the initial probabilities

In [5]:
hidden_states = ['not mastered', 'mastered']
initial_state_prob = [0.9, 0.1]

In [6]:
state_space = pd.Series(initial_state_prob, index=hidden_states, name='states')
print state_space
print "Check - should sum to 1.0: ", state_space.sum()

not mastered    0.9
mastered        0.1
Name: states, dtype: float64
Check - should sum to 1.0:  1.0


Create initial hidden state transition matric, MxM where M is number of hidden states

In [7]:
transmatrix_df = pd.DataFrame(columns=hidden_states, index=hidden_states)
transmatrix_df.loc[hidden_states[0]] = [0.5, 0.5]
transmatrix_df.loc[hidden_states[1]] = [0.15, 0.85]
print transmatrix_df
a = transmatrix_df.values
print a, a.shape
print(transmatrix_df.sum(axis=1))

             not mastered mastered
not mastered          0.5      0.5
mastered             0.15     0.85
[[0.5 0.5]
 [0.15 0.85]] (2, 2)
not mastered    1.0
mastered        1.0
dtype: float64


Set up the observable states and transition probabilities
observation probability matrix is MxO where M is number of states, O is number of observations

In [8]:
observable_states = ['took content 1', 'took content 2', 'played game', 'test score < 85', 'test score >= 85']

In [9]:
obsprobmatrix_df = pd.DataFrame(columns=observable_states, index=hidden_states)
obsprobmatrix_df.loc[hidden_states[0]] = [0.2, 0.1, 0.3, 0.3, 0.1]
obsprobmatrix_df.loc[hidden_states[1]] = [0.05, 0.1, 0.1, 0.05, 0.7]

print obsprobmatrix_df

b = obsprobmatrix_df.values
print b, b.shape

print(obsprobmatrix_df.sum(axis=1))

             took content 1 took content 2 played game test score < 85  \
not mastered            0.2            0.1         0.3             0.3   
mastered               0.05            0.1         0.1            0.05   

             test score >= 85  
not mastered              0.1  
mastered                  0.7  
[[0.2 0.1 0.3 0.3 0.1]
 [0.05 0.1 0.1 0.05 0.7]] (2, 5)
not mastered    1.0
mastered        1.0
dtype: float64


Run an observed sequence of states through the viterbi alg to see most likely hidden state path
played game, failed test, content 1, failed test, content 2, played game, passed test

In [14]:
observations_map = {'took content 1':0, 'took content 2':1, 'played game':2, 'test score < 85':3, 'test score >= 85':4}
observations = np.array([2, 3, 0, 3, 1, 2, 4])

inv_observations_map = dict((v,k) for k, v in observations_map.items())
observation_sequence = [inv_observations_map[v] for v in list(observations)]

print pd.DataFrame(np.column_stack([observations, observation_sequence]), columns=['Obs Code', 'Obs Seq'])


  Obs Code           Obs Seq
0        2       played game
1        3   test score < 85
2        0    took content 1
3        3   test score < 85
4        1    took content 2
5        2       played game
6        4  test score >= 85


Define the Viterbi alg. for state path evaluation.  See http://www.blackarbs.com/blog/introduction-hidden-markov-models-python-networkx-sklearn/2/9/2017 and assoc reference.


In [13]:
def viterbi(pi, a, b, obs):
    
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]
    
    # init blank path
    path = np.zeros(T)
    # delta --> highest probability of any path that reaches state i
    delta = np.zeros((nStates, T))
    # phi --> argmax by time step for each state
    phi = np.zeros((nStates, T))
    
    # init delta and phi 
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0

    print('\nStart Walk Forward\n')    
    # the forward algorithm extension
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
            print('s={s} and t={t}: phi[{s}, {t}] = {phi}'.format(s=s, t=t, phi=phi[s, t]))
    
    # find optimal path
    print('-'*50)
    print('Start Backtrace\n')
    #print "phi: ", phi
    path[T-1] = np.argmax(delta[:, T-1])
    #p('init path\n    t={} path[{}-1]={}\n'.format(T-1, T, path[T-1]))
    for t in range(T-2, -1, -1):
        #debugging index error in extracting path
        #print "t and t+1: ", t, t+1
        phi_row = int(path[t+1])
        phi_col = t+1
        #print "phi indices: ", phi_row, phi_col
        #print "phi @ indices: ", phi[0, 6]
        
        path[t] = phi[phi_row, phi_col]
        #p(' '*4 + 't={t}, path[{t}+1]={path}, [{t}+1]={i}'.format(t=t, path=path[t+1], i=[t+1]))
        print('path[{}] = {}'.format(t, path[t]))
        
    return path, delta, phi


Run Vitiberi alg. on the observed sequence

In [68]:
path, delta, phi = viterbi(initial_state_prob, a, b, observations)


Start Walk Forward

s=0 and t=1: phi[0, 1] = 0.0
s=1 and t=1: phi[1, 1] = 0.0
s=0 and t=2: phi[0, 2] = 0.0
s=1 and t=2: phi[1, 2] = 0.0
s=0 and t=3: phi[0, 3] = 0.0
s=1 and t=3: phi[1, 3] = 0.0
s=0 and t=4: phi[0, 4] = 0.0
s=1 and t=4: phi[1, 4] = 0.0
s=0 and t=5: phi[0, 5] = 0.0
s=1 and t=5: phi[1, 5] = 1.0
s=0 and t=6: phi[0, 6] = 0.0
s=1 and t=6: phi[1, 6] = 0.0
--------------------------------------------------
Start Backtrace

phi:  [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]
path[5] = 0.0
path[4] = 0.0
path[3] = 0.0
path[2] = 0.0
path[1] = 0.0
path[0] = 0.0


In [70]:
print "most likely state path: ", path
print "delta: ", delta
print "phi", phi

most likely state path:  [0. 0. 0. 0. 0. 0. 1.]
delta:  [[2.7000000e-01 4.0500000e-02 4.0500000e-03 6.0750000e-04 3.0375000e-05
  4.5562500e-06 2.2781250e-07]
 [1.0000000e-02 6.7500000e-03 1.0125000e-03 1.0125000e-04 3.0375000e-05
  2.5818750e-06 1.5946875e-06]]
phi [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]


In [76]:
def print_state_path(state_map, path):
    state_path = [state_map[v] for v in path]
    results_df = pd.DataFrame().assign(Observation=observation_sequence).assign(Likely_Path=state_path)
    print results_df

In [79]:
state_map = {0:'Not mastered', 1:'Mastered'}

print_state_path(state_map, path)

        Observation   Likely_Path
0       played game  Not mastered
1   test score < 85  Not mastered
2    took content 1  Not mastered
3   test score < 85  Not mastered
4    took content 2  Not mastered
5       played game  Not mastered
6  test score >= 85      Mastered


Run another set of observations

In [81]:
observations_map = {'took content 1':0, 'took content 2':1, 'played game':2, 'test score < 85':3, 'test score >= 85':4}
observations = np.array([0, 1, 2, 4, 2])

inv_observations_map = dict((v,k) for k, v in observations_map.items())
observation_sequence = [inv_observations_map[v] for v in list(observations)]

print pd.DataFrame(np.column_stack([observations, observation_sequence]), columns=['Obs Code', 'Obs Seq'])

  Obs Code           Obs Seq
0        0    took content 1
1        1    took content 2
2        2       played game
3        4  test score >= 85
4        2       played game


In [82]:
path, delta, phi = viterbi(initial_state_prob, a, b, observations)


Start Walk Forward

s=0 and t=1: phi[0, 1] = 0.0
s=1 and t=1: phi[1, 1] = 0.0
s=0 and t=2: phi[0, 2] = 0.0
s=1 and t=2: phi[1, 2] = 1.0
s=0 and t=3: phi[0, 3] = 0.0
s=1 and t=3: phi[1, 3] = 0.0
s=0 and t=4: phi[0, 4] = 1.0
s=1 and t=4: phi[1, 4] = 1.0
--------------------------------------------------
Start Backtrace

phi:  [[0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 1.]]
path[3] = 1.0
path[2] = 0.0
path[1] = 0.0
path[0] = 0.0


In [83]:
print "most likely state path: ", path
print "delta: ", delta
print "phi", phi

most likely state path:  [0. 0. 0. 1. 1.]
delta:  [[1.80000e-01 9.00000e-03 1.35000e-03 6.75000e-05 2.12625e-05]
 [5.00000e-03 9.00000e-03 7.65000e-04 4.72500e-04 4.01625e-05]]
phi [[0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 1.]]


In [84]:
print_state_path(state_map, path)

        Observation   Likely_Path
0    took content 1  Not mastered
1    took content 2  Not mastered
2       played game  Not mastered
3  test score >= 85      Mastered
4       played game      Mastered
